## Boto3, Lambda, CloudWatch

001에서 했던 crontab의 최대 문제는, 서버를 계속 켜놔야 한다는 것. <br>
lambda, cloudwatch, boto3를 이용해서 이 문제를 해결할 수 있음. 

- boto3 : instant start와 stop을 파이썬 코드로 할 수 있게 만들어 줌. 
    - 하루에 5분 12시 - 12시 5분에만 크롤링을 하는데 24시간 켜 놓을 필요가 없음. 
    - 다른 서버에서 이 서버를 12시에 켜주고 12시 5분에 꺼주면 내 서버는 하루에 5분만 켜있게 되는 것. 
    - 다른 서버에서는 boto3를 crontab에 올려서 12시, 12시 5분에 키고 끄는 명령만 보내면 됨. 
    - 서버 키면서 동시에, slack.py 같은것 실행시키면 됨.
    - 근데 여기서도 문제가 발생하는게, 이 어떤 서버에서는 boto3를 실행시켜 놔야 하는 것. 
    
- lambda: server가 없이 python code를 실행시켜주는 aws service. 
    - 즉, boto3 start&stop을 lambda에 작성을 해놓는다. 
    - cloudwatch는 lambda에 올라가 있는 코드가 실행되게 해주는 trigger 역할을 해준다. 
    - cloudwatch를 통해, lambda에 올라가 있는 boto3 코드를 실행해 주면 되는 것. 

### boto3

- 일단 이것을 해주려면, 서버가 2개가 있어야 함. 컨트롤 해주는 서버, 크롤링 코드를 실행시키는 서버. 
- 인스턴스 하나 더 생성. 
- AWS - EC2 - AMI 
- `public image` - `소유자` - `035683184441` 검색(검색불가)
- 나는 내 기존 환경 이미지 Private AMI 로 만들어놓고, 복사함. 

- 그 이미지 가지고 인스턴스 생성. 
- 다만, 그때 보안그룹 구성. 
    - 새로 생성하면, 22번 포트만 열려 있어서 기존 보안그룹으로 다시 선택하는 것. 
    - 새 보안 그룹이 디폴트로 되어 있음. 기존 보안 그룹중에 내가 원래 쓰던 보안그룹 선택. 
    - 검토 및 시작. 
- keypair는 내가 기존에 쓰던 keypair 선택하면 됨.     
- 인스턴스 만든 후에는, 이름 내 맘대로 변경해도 됨. such as crawling-server
- ip생성되면, 그 ip로 접속 하면 된다. 
    - 터미널 새로 열고
    - `ssh -i /Users/sanghyuk/Mirror/aws_setting/<암호route> ubuntu@<ip>`
    - 이때, 만약 내 원래 서버에 키파일 올려 놓으면 서버에서 또 다른 서버로도 접속 당연히 가능. 명령도 동일

#### In Action

##### 새로 생성한 instance - crawling server

- 지금 새로 만든 서버를 crawling server로 쓸 것. 
- `vi slack.py` 열어서 위에 코드 복사하고 저장. 
- slack.py가 잘 동작하는지 test
    - python3 weather.py
- crontab에 이 코드를 넣어주자. 
    - crontab -l : 설정 보기 
    - crontab -e : setting
    - `*` 쓰는 부분에 `@reboot` 이라고 작성하면 부팅 될 때 실행된다. 
    - `@reboot python3 slack.py`
    - 이렇게 쓰면, 서버가 꺼졌다가 딱 켜졌을 때 이 코드가 실행되는 것. 
- 테스트 하려면, 
    - 작업-상태-중지(종료하면 절대 안된다)
    - 시작 
    - 이렇게 재부팅 하고 slack으로 메세지 날아왔는지 check